#### Main Goal of Project: Prediction, not interpretation
##### looking into features that are more important for the most winningest teams, which is another project idea that I would like to explore, is not that point of thie project. Rather, predicting final season points for teams is the objetive. Thus, there is less focus on model evaluation or colinearity. Instead, it is more important to build a model that predicts number of points (since relative points translate to table position i.e. relativity) well for a team based on their season-to-season data, without considering the 'quality' of the model too much.

In [12]:
import numpy as np
import pandas as pd
import xgboost as xgb
import plotly
import sklearn

In [2]:
epldf = pd.read_csv('EPL Standings 2000-2022.csv')

In [3]:
epldf.tail(20)

,Season,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts,Qualification or relegation
420,2021-22,1,Manchester City,38,29,6,3,99,26,73,93,Qualification for the Champions League group s...
421,2021-22,2,Liverpool,38,28,8,2,94,26,68,92,Qualification for the Champions League group s...
422,2021-22,3,Chelsea,38,21,11,6,76,33,43,74,Qualification for the Champions League group s...
423,2021-22,4,Tottenham Hotspur,38,22,5,11,69,40,29,71,Qualification for the Champions League group s...
424,2021-22,5,Arsenal,38,22,3,13,61,48,13,69,Qualification for the Europa League group stag...
425,2021-22,6,Manchester United,38,16,10,12,57,57,0,58,Qualification for the Europa League group stag...
426,2021-22,7,West Ham United,38,16,8,14,60,51,9,56,Qualification for the Europa Conference League...
427,2021-22,8,Leicester City,38,14,10,14,62,59,3,52,Not Applicable
428,2021-22,9,Brighton & Hove Albion,38,12,15,11,42,44,-2,51,Not Applicable
429,2021-22,10,Wolverhampton Wanderers,38,15,6,17,38,43,-5,51,Not Applicable


In [4]:
epldf.shape

(440, 12)

In [5]:
X = epldf[['W','D','L','GF','GA','GD']]
Y = epldf['Pts']

In [6]:
X_train = X.iloc[:420]
y_train = Y.iloc[:420]
X_test = X.iloc[-20:]
y_test = Y.iloc[-20:]


In [7]:
model = xgb.XGBRegressor()

In [8]:
from sklearn.model_selection import GridSearchCV

# set up search grid
param_grid = {"max_depth":    [4, 5,6],
              "learning_rate": [0.01, 0.015,0.02]}

# try out every combination of the above values
search = GridSearchCV(model, param_grid, cv=5).fit(X_train, y_train)

print("The best parameters are: ", search.best_params_)

The best parameters are:  {'learning_rate': 0.02, 'max_depth': 4}


In [9]:
model=xgb.XGBRegressor(learning_rate = search.best_params_["learning_rate"],
                           max_depth     = search.best_params_["max_depth"],)

model.fit(X_train, y_train)

predictions = model.predict(X_test)



In [10]:
predictions

array([76.959785, 76.18207 , 63.85281 , 63.070343, 61.440582, 50.738384,
       50.131256, 45.092926, 44.26497 , 43.64818 , 41.13605 , 40.886757,
       40.281147, 38.814606, 34.756184, 34.292862, 32.103348, 31.391779,
       18.8686  , 20.650587], dtype=float32)

In [19]:
import plotly.graph_objects as go

In [30]:
position = epldf.iloc[:20]['Pos']

fig = go.Figure()
fig.add_trace(go.Bar(x=position,
                y=predictions,
                name='Predicted',
                marker_color='rgb(55, 83, 109)'
                ))
fig.add_trace(go.Bar(x=position,
                y=epldf.iloc[:20]['Pts'],
                name='Actual',
                marker_color='rgb(26, 118, 255)'
                ))

fig.update_layout(
    title='Predicted vs Actual Points (2021-2022 season)',
    xaxis=dict(
        title='Position',
        tickfont_size=14,
    ),
    yaxis=dict(
        title='Points',
        titlefont_size=16,
        tickfont_size=14
    ),
    legend=dict(
        x=1.00,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.show()